In [2]:
import pandas as pd
from tqdm import tqdm
from IPython.display import display 
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain

from utils import evaluate, ColBERTv2


train = [('Who produced the album that included a re-recording of "Lithium"?', ['Butch Vig']),
         ('Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?', ['Kevin Greutert']),
         ('The heir to the Du Pont family fortune sponsored what wrestling team?', ['Foxcatcher', 'Team Foxcatcher', 'Foxcatcher Team']),
         ('In what year was the star of To Hell and Back born?', ['1925']),
         ('Which award did the first book of Gary Zukav receive?', ['U.S. National Book Award', 'National Book Award']),
         ('What city was the victim of Joseph Druces working in?', ['Boston, Massachusetts', 'Boston']),]

dev = [('Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?', ['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow']),
       ('What documentary about the Gilgo Beach Killer debuted on A&E?', ['The Killing Season']),
       ('Right Back At It Again contains lyrics co-written by the singer born in what city?', ['Gainesville, Florida', 'Gainesville']),
       ('What year was the party of the winner of the 1971 San Francisco mayoral election founded?', ['1828']),
       ('Which author is English: John Braine or Studs Terkel?', ['John Braine']),
       ('Anthony Dirrell is the brother of which super middleweight title holder?', ['Andre Dirrell']),
       ('In which city is the sports nutrition business established by Oliver Cookson based ?', ['Cheshire', 'Cheshire, UK']),
       ('Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.', ['February 13, 1980']),
       ('Kyle Moran was born in the town on what river?', ['Castletown', 'Castletown River']),
       ("What is the name of one branch of Robert D. Braun's speciality?", ['aeronautical engineering', 'astronautical engineering', 'aeronautics', 'astronautics']),
       ("Where was the actress who played the niece in the Priest film born?", ['Surrey', 'Guildford, Surrey']),
       ('Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.', ['Portrait of a Marriage']),
       ('What year was the father of the Princes in the Tower born?', ['1442'])]


train = [{'question': q, 'answer': a[0]} for q, a in train]
dev = [{'question': q, 'answers': a} for q, a in dev]

In [3]:
llm = OpenAI(temperature=0.9)
rm = ColBERTv2('http://ec2-44-228-128-229.us-west-2.compute.amazonaws.com:8893/api/search')

# 1 - Vanilla GPT-3.5

In [12]:
example_question_answer_template = """
Question: {question}
Answer: {answer}
"""
example_prompt = PromptTemplate(
    input_variables=['question', 'answer'],
    template=example_question_answer_template,
)

prefix = """
Answer questions with short factoid answers.

---

Follow the following format.

Question: $[the question to be answered]
Answer: $[a short factoid answer, often between 1 and 5 words]

---
"""

few_shot_prompt = FewShotPromptTemplate(
    examples=train,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix="\nQuestion: {question}\nAnswer:",
    input_variables=['question'],
    example_separator=''
)

chain = LLMChain(llm=llm, prompt=few_shot_prompt)

def run_vanilla_chain(example):
    return chain.run(question=example['question'])

In [4]:
evaluate(run_vanilla_chain, dev)

  0%|          | 0/13 [00:00<?, ?it/s]


RateLimitError: You exceeded your current quota, please check your plan and billing details.

# 2 - Retrieve then read

In [11]:
prefix = """
Answer questions with short factoid answers.
"""

suffix = """
---

Follow the following format.

Context:
$[sources that may contain relevant content]

Question: $[the question to be answered]

Answer: $[a short factoid answer, often between 1 and 5 words]

---

Context:
{context}

Question: {question}

Answer:
"""

retrieval_prompt = FewShotPromptTemplate(
    examples=train,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=['context', 'question'],
    example_separator=''
)

chain = LLMChain(llm=llm, prompt=retrieval_prompt)

def run_retrieval_chain(example):
    context = rm(example['question'])[0]
    return chain.run(context=context, question=example['question'])

In [14]:
evaluate(run_retrieval_chain, dev)

100%|██████████| 13/13 [00:13<00:00,  1.07s/it]

Correct: 5 / 13


,question,answers,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"[E. L. Doctorow, E.L. Doctorow, Doctorow]",E.L. Doctorow,✅
1,What documentary about the Gilgo Beach Killer debuted on A&E?,[The Killing Season],The Killing Season,✅
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"[Gainesville, Florida, Gainesville]","Orlando, Florida",❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,[1828],1936,❌
4,Which author is English: John Braine or Studs Terkel?,[John Braine],Studs Terkel,❌
5,Anthony Dirrell is the brother of which super middleweight title holder?,[Andre Dirrell],Andre Dirrell,✅
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"[Cheshire, Cheshire, UK]","Manchester, UK",❌
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"[February 13, 1980]",5 August 1862,❌
8,Kyle Moran was born in the town on what river?,"[Castletown, Castletown River]",Liffey River,❌
9,What is the name of one branch of Robert D. Braun's speciality?,"[aeronautical engineering, astronautical engineering, aeronautics, astronautics]",Jet Propulsion,❌


# 3 - Retrieve then read with self-consistency

In [39]:
from collections import Counter

prefix = """
Answer questions with short factoid answers.
"""

suffix = """
---

Follow the following format.

Context:
$[sources that may contain relevant content]

Question: $[the question to be answered]

Rationale: Let's think step by step. $[a step-by-step deduction that identifies the correct response, which will be provided below]

Answer: $[a short factoid answer, often between 1 and 5 words]

---

Context:
{context}

Question: {question}

Rationale: Let's think step by step."""

rationale_prompt = FewShotPromptTemplate(
    examples=train,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=['context', 'question'],
    example_separator=''
)

chain = LLMChain(llm=llm, prompt=rationale_prompt)

def run_rationale_chain(example, n=20):
    context = rm(example['question'], k=5)
    context = '\n'.join([f'[{i+1}] {c}' for i, c in enumerate(context)])

    # Self consistency
    answers = []
    for i in range(n):
        completion = chain.run(context=context, question=example['question'])
        last_line = completion.split('\n')[-1].strip()
        answer = last_line.split(':')[-1].strip()
        answers.append(answer)

    return Counter(answers).most_common(1)[0][0]

In [40]:
evaluate(run_rationale_chain, dev)

  8%|▊         | 1/13 [01:13<14:47, 73.96s/it]

['Physician', 'E.L. Doctorow', 'Physician', 'E. L. Doctorow', 'E.L. Doctorow', 'Julia Peterkin', 'E. L. Doctorow', 'Doctorow', 'Physician', 'E.L. Doctorow', 'Doctorow', 'Julia Peterkin', 'E. L. Doctorow', 'Physician', 'Physician', 'E. L. Doctorow', 'E. L. Doctorow', 'Doctorow', 'Physician', 'E. L. Doctorow']


 15%|█▌        | 2/13 [02:14<12:07, 66.14s/it]

['The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season', 'The Killing Season']


 23%|██▎       | 3/13 [03:35<12:10, 73.02s/it]

...

 92%|█████████▏| 12/13 [14:50<01:19, 79.62s/it]

['Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', '"Portrait of a Marriage"', 'Portrait of a Marriage.', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage', 'Portrait of a Marriage']


100%|██████████| 13/13 [16:08<00:00, 74.49s/it]

['1442', '1462', '1312', '1312', '1455', 'c. 1455', 'c. 1455', '1442', '1442', '1442', '1442', '1455', 'c. 1455', '1442', '1442', '1442', '1340', '1442', '1455', 'c. 1455']
Correct: 8 / 13


,question,answers,prediction,correct
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,"[E. L. Doctorow, E.L. Doctorow, Doctorow]",Physician,❌
1,What documentary about the Gilgo Beach Killer debuted on A&E?,[The Killing Season],The Killing Season,✅
2,Right Back At It Again contains lyrics co-written by the singer born in what city?,"[Gainesville, Florida, Gainesville]","Ocala, Florida",❌
3,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,[1828],1828,✅
4,Which author is English: John Braine or Studs Terkel?,[John Braine],John Braine,✅
5,Anthony Dirrell is the brother of which super middleweight title holder?,[Andre Dirrell],Andre Dirrell,✅
6,In which city is the sports nutrition business established by Oliver Cookson based ?,"[Cheshire, Cheshire, UK]","Cheshire, UK",✅
7,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"[February 13, 1980]",27 September 1974,❌
8,Kyle Moran was born in the town on what river?,"[Castletown, Castletown River]",Castletown River,✅
9,What is the name of one branch of Robert D. Braun's speciality?,"[aeronautical engineering, astronautical engineering, aeronautics, astronautics]",Aerospace engineering,❌


# 4 - Multi-hop retrieval

In [4]:
search_retrieval_template_first_hop = """
Write a search query that will help answer a complex question.

---

Follow the following format.

Question: $[the question to be answered]
Rationale: Let's think step by step. To answer this question, we first need to find out $[the missing information]
Search Query: $[a simple question for seeking the missing information]

---

Question: {question}
Rationale: Let's think step by step. To answer this question, we first need to find out"""

first_hop_prompt = PromptTemplate(
    input_variables=['question'],
    template=search_retrieval_template_first_hop,
)

chain = LLMChain(llm=llm, prompt=first_hop_prompt)
completion_first_hop = chain.run(dev[2]['question'])

In [5]:
completion_first_hop

' which singer co-wrote the lyrics to "Right Back At It Again".\nSearch Query: Who co-wrote the lyrics to "Right Back At It Again"?'

In [6]:
retrieval_question_first_hop = completion_first_hop.split('\n')[-1].split(':')[-1].strip()
retrieval_question_first_hop

'Who co-wrote the lyrics to "Right Back At It Again"?'

In [8]:
def format_context_for_prompt(context):
    """
    Format and enumerate a list of context strings for use in a prompt.
    """
    return '\n'.join([f'[{i+1}] {c}' for i, c in enumerate(context)])


def extract_last_line(completion, remove_prefix=True):
    """
    Extract the last line of a completion, optionally removing the prefix.
    """
    last_line = completion.split('\n')[-1].strip()
    if remove_prefix:
        last_line = last_line.split(':')[-1].strip()
    return last_line


search_retrieval_template_second_hop = """
Write a search query that will help answer a complex question.

---

Follow the following format.

Context:
$[sources that may contain relevant content]

Question: $[the question to be answered]
Rationale: Let's think step by step. Based on the context, we have learned the following. $[information from the context that provides useful clues]
Search Query: $[a simple question for seeking the missing information]

---

Context:
{context}

Question: {question}
Rationale: Let's think step by step. Based on the context, we have learned the following."""

context = rm(retrieval_question_first_hop, k=2)
context = format_context_for_prompt(context)

second_hop_prompt = PromptTemplate(
    input_variables=['context', 'question'],
    template=search_retrieval_template_second_hop,
)

print(second_hop_prompt.format(context=context, question=dev[2]['question']))


Write a search query that will help answer a complex question.

---

Follow the following format.

Context:
$[sources that may contain relevant content]

Question: $[the question to be answered]
Rationale: Let's think step by step. Based on the context, we have learned the following. $[information from the context that provides useful clues]
Search Query: $[a simple question for seeking the missing information]

---

Context:
[1] Right Back at It Again | Right Back at It Again "Right Back at It Again" is the second track and the first single from A Day to Remember's fifth album, "Common Courtesy" (2013). In October 20, 2015, the song was featured in Activision rhythm-music game, "". Vocalist, Jeremy McKinnon wrote the lyrics, while the music was written by McKinnon, former guitarist Tom Denney, guitarist Neil Westfall and producer Andrew Wade. "Right Back at It Again" almost wasn't included on the album as it was one of the excess songs the band had recorded, "we realised that it soun

In [9]:
chain = LLMChain(llm=llm, prompt=second_hop_prompt)
completion_second_hop = chain.run(context=context, question=dev[2]['question'])
completion_second_hop

' The singer of Right Back At It Again is Jeremy McKinnon and he co-wrote the lyrics.\n\nSearch Query: "Jeremy McKinnon" birthplace'

In [10]:
retrieval_question_second_hop = completion_second_hop.split('\n')[-1].split(':')[-1].strip()
retrieval_question_second_hop

'"Jeremy McKinnon" birthplace'

In [13]:
prefix = """
Answer questions with short factoid answers.
"""

suffix = """
---

Follow the following format.

Context:
$[sources that may contain relevant content]

Question: $[the question to be answered]

Rationale: Let's think step by step. $[a step-by-step deduction that identifies the correct response, which will be provided below]

Answer: $[a short factoid answer, often between 1 and 5 words]

---

Context:
{context}

Question: {question}

Rationale: Let's think step by step."""

context = rm(retrieval_question_second_hop, k=2)
context = format_context_for_prompt(context)

rationale_prompt = FewShotPromptTemplate(
    examples=train,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=['context', 'question'],
    example_separator=''
)

chain = LLMChain(llm=llm, prompt=rationale_prompt)
final_completion = chain.run(context=context, question=dev[2]['question'])

In [14]:
final_completion

" First, let's consider the provided context. We are told that Jeremy McKinnon is the singer in A Day to Remember, which is a band he founded. We are also informed that he is an American musician and record producer. We can deduce from this that the song Right Back At It Again is a song by A Day to Remember, and that Jeremy McKinnon would have co-written the lyrics. Then, we check the first source to see where Jeremy McKinnon was born - Gainesville, Florida. \n\nAnswer: Gainesville, Florida"

In [15]:
answer = final_completion.split('\n')[-1].split(':')[-1].strip()
answer

'Gainesville, Florida'